In [1]:
import os

import torch


In [2]:
os.chdir("..")

In [3]:
directory = "rezultati testiranja 26-5"

In [30]:
# list files in the directory and keep only those that start with "error_analysis_"

files = [f for f in os.listdir(directory) if f.startswith("error_analysis_") and f not in ["error_analysis_20250523_173550.pt","error_analysis_20250523_191132.pt","error_analysis_20250523_200854.pt","error_analysis_20250523_213529.pt","error_analysis_20250523_231815.pt"]]
files.sort()

In [31]:
files

['error_analysis_20250524_024423.pt',
 'error_analysis_chatgpt_batch_structured.pt',
 'error_analysis_chatgpt_individual_plain.pt',
 'error_analysis_chatgpt_individual_structured.pt',
 'error_analysis_gemma-3-12b-it-unsloth-bnb-4bit-finetuned.pt',
 'error_analysis_local_gemma_batch_structured.pt',
 'error_analysis_local_gemma_individual_plain.pt',
 'error_analysis_local_gemma_individual_structured.pt']

In [32]:
for file in files:
    f = os.path.join(directory, file)
    run_name = file[len("error_analysis_"):].split(".")[0]
    content = torch.load(f)
    print(content)
    break

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [33]:
# '101', '108', '113'
selected_doc = '101'

In [34]:
content.keys()

dict_keys(['727', '78', '206', '687', '792', '781', '207', '196', '677', '118', '66', '667', '446', '368', '222', '21', '436', '108', '32', '516', '88', '71', '601', '392', '397', '333', '262', '221', '283', '53', '487', '73', '113', '132', '61', '431', '288', '296', '817', '33', '101', '552', '398', '742'])

In [35]:
content[selected_doc]

{'events': [{'event_id': 'E0',
   'event_char_start': tensor(1),
   'event_char_end': tensor(10),
   'pred_start': (48135724.561187744, 48135664.561187744, 48135784.561187744),
   'pred_end': (48135460.07594299, 48135400.07594299, 48135520.07594299),
   'pred_duration': (48135223.43084717, 48135163.43084717, 48135283.43084717),
   'gold_start': (48135600.0, 48135420.0, 48135960.0),
   'gold_end': (48135605.0, 48135422.0, 48135970.0),
   'gold_duration': (5.0, 2.0, 10.0)},
  {'event_id': 'E18',
   'event_char_start': tensor(1024),
   'event_char_end': tensor(1053),
   'pred_start': (48131080.743652344, 48131020.743652344, 48131140.743652344),
   'pred_end': (48134535.81152344, 48134475.81152344, 48134595.81152344),
   'pred_duration': (48136283.24169922, 48136223.24169922, 48136343.24169922),
   'gold_start': (48066480.0, 48063600.0, 48069360.0),
   'gold_end': (48066500.0, 48063615.0, 48069385.0),
   'gold_duration': (20.0, 15.0, 25.0)},
  {'event_id': 'E19',
   'event_char_start': ten

In [36]:
def annotate_events(summary, events):
    event_starts = [int(event["event_char_start"]) for event in events]
    event_ends = [int(event["event_char_end"]) for event in events]
    for i in range(len(events)):
        start = event_starts[i]
        end = event_ends[i]
        label = events[i]["event_id"]
        summary = summary[:start] + f"[{label}]" + summary[start:end] + f"[/{label}]" + summary[end:]
        offset = len(f"[{label}]") + len(f"[/{label}]")
        for j in range(i + 1, len(events)):
            if event_starts[j] >= start:
                event_starts[j] += offset
                event_ends[j] += offset
    return summary

In [37]:
import datetime

BASE_DATETIME = datetime.datetime(1900, 1, 1, 0, 0, 0)
def convert_minutes_to_datetime(minutes_since_base, duration=False):
    """Converts minutes since BASE_DATETIME to a datetime object."""
    delta = datetime.timedelta(minutes=int(minutes_since_base))
    if duration:
        return delta
    else:
        return str(BASE_DATETIME + delta)

In [38]:
def analyze_document(content, analyzed_doc, predictions_dict, experiment_label=None):
    text = str(content[analyzed_doc]["text"])
    events = content[analyzed_doc]["events"]
    annotated_text = annotate_events(text, events)
    # print(events[0])
    if "gold_start" not in predictions_dict:
        predictions_dict["gold_start"] = {}
        for event in events:
            predictions_dict["gold_start"][event["event_id"]] = convert_minutes_to_datetime(event["gold_start"][0])
    if "gold_end" not in predictions_dict:
        predictions_dict["gold_end"] = {}
        for event in events:
            predictions_dict["gold_end"][event["event_id"]] = convert_minutes_to_datetime(event["gold_end"][0])
    if "gold_duration" not in predictions_dict:
        predictions_dict["gold_duration"] = {}
        for event in events:
            predictions_dict["gold_duration"][event["event_id"]] = convert_minutes_to_datetime(event["gold_duration"][0])

    predictions_dict[experiment_label + "_start"] = {}
    predictions_dict[experiment_label + "_end"] = {}
    predictions_dict[experiment_label + "_duration"] = {}

    for event in events:
        predictions_dict[experiment_label + "_start"][event["event_id"]] = convert_minutes_to_datetime(event["pred_start"][0] if isinstance(event["pred_start"], tuple) else event["pred_start"])
        predictions_dict[experiment_label + "_end"][event["event_id"]] = convert_minutes_to_datetime(event["pred_end"][0] if isinstance(event["pred_end"], tuple) else event["pred_end"])
        predictions_dict[experiment_label + "_duration"][event["event_id"]] = convert_minutes_to_datetime(event["pred_duration"][0] if isinstance(event["pred_duration"], tuple) else event["pred_duration"])

    return annotated_text
# analyze_document(selected_doc, {})

In [39]:
import pandas as pd

predictions = {}
first_run = True
for file in files:
    f = os.path.join(directory, file)
    run_name = file[len("error_analysis_"):].split(".")[0]
    content = torch.load(f, weights_only=False)
    # print(content)
    text = analyze_document(content=content, analyzed_doc=selected_doc, predictions_dict=predictions, experiment_label=run_name)
    if first_run:
        print(text)
    first_run = False
df = pd.DataFrame(predictions)
df.to_excel("error_analysis_predictions.xlsx")


[E0]Admis[E0]sion[[E0]/E0][[E0]/E0][[E0]/E0][/E0][/E0] Date :
07/10/1991
[E111]Dischar[E111]ge[/E11[E111]1][/E11[E111]1][/E11[E111]1][/E111][/E111] Date :
07/18/1991
PROCEDURES :
[E112]Thorace[E112]ntesis[[E112]/E112][[E112]/E112][[E112]/E112][/E112][/E112] was performed on 7-12-91 .
REASON FOR ADMISSION :
The patient is an 85-year-old white male with a history of [E113]ischemi[E113]c bowel[E113][/E113][E113][/E113][E113][/E113][/E113][/E113] status post recent [E101]admissi[E101]on[/E10[E101]1][/E10[E101]1][/E10[E101]1][/E101][/E101] for [E114]uroseps[E114]is[/E11[E114]4][/E11[E114]4][/E11[E114]4][/E114][/E114] and [E1]C. di[E1]ffici[E1]le co[E1]litis[E1][/E1][/E1][/E1][/E1][/E1] .
He returns from [E118]the nur[E118]sing ho[E118]me[/E11[E118]8][/E11[E118]8][/E118][/E118][/E118] with [E115]fever[/[E115]E115][/[E115]E115][/[E115]E115][/[E115]E115][/E115] , [E116]leukocy[E116]tosis[/[E116]E116][/[E116]E116][/[E116]E116][/E116][/E116] , and [E117]azotemi[E117]a[/E117[E117]][/E117[E117]][